# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [ ]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('dayofweek', axis=1), df['dayofweek'],
    test_size=0.2, random_state=21, stratify=df['dayofweek']
    )

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
svc = SVC(probability=True, random_state=21)

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]
              }

gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [ ]:
results.head(7)

## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
tree = DecisionTreeClassifier(random_state=21)

In [ ]:
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None]
            }

gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [ ]:
results

## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
frt = RandomForestClassifier(random_state=21)

In [ ]:
param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]}

gs = GridSearchCV(frt, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [ ]:
results

## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
grid = list(ParameterGrid(param_grid))

In [ ]:
data = []

for params in tqdm(grid):
    # print(params)
    # print(1)
    d = {}
    estimator = RandomForestClassifier(**params)
    sc = cross_val_score(estimator, X_train, y_train, cv=5, n_jobs=-1)
    # print(sc)
    d = {**params, 'mean_accuracy': np.mean(sc), 'std_accuracy': np.std(sc)}
    data.append(d)

In [ ]:
results_1 = pd.DataFrame(data)
results_1 = results_1.sort_values('mean_accuracy', ascending=False)
results_1

## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [ ]:
rfc = RandomForestClassifier(n_estimators=50, max_depth=28, criterion='gini', class_weight=None, random_state=21)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)